# **Note for basic concepts for tensorflow**
---
<br>

## Import the basic packages
- Can set tensorflow to eager mode to run faster on small functions

In [ ]:
import tensorflow as tf 
# tf.enable_eager_execution()

## Try use tensors
- tf.zeros: all 0
- tf.ones: all 1
- tf.fill: all target number
- tf.constant(\[element_list\])
- tf.mutmul(a, b): multiplication
- Use Variable to create tensor with target distribution, set a seed and the w will remain the same every time fire up the programe

In [ ]:
a = tf.constant([[1.,3.]])
b = tf.constant([[4.],[2.]])
# tensor multiplication, need to check if the two tensors can be multiplied.
y = tf.matmul(a, b) 
# excute the computation using tensorflow session
with tf.Session() as sess:
    sess.run(y)
    print(y)
# use Variable to create tensor
w = tf.Variable(tf.random_normal([2, 3], stddev=2, mean=0, seed=1))

## Init the gloable variable
- define the input data type, shape (dimensions, None means any natural number)
- feed data into model, the data shape should be compatiable with the pre-defined one.

In [ ]:
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    in_data = tf.placeholder(tf.float32, shape=(None, 2))
    sess.run(y, feed_dict={in_data:[[.5, .6],[.4, .4],[.6, .9]]})

## The hperparameter settings
- loss function: MSE 均方误差, CrossEntropyLoss 等
- optimizer function: GSD, Momemtum, Adam, RSMprop 等
- activation functions: sigmoid, tanh \[Linear\] / ReLU \[Non-linear\]
- batch training: use batch size to define how many samples would be considered during one step of optimization.
- we also use sigmoid and softmax for two / multi classes classification tasks to turn the predictions into probabilities.
- learning rate: the decay learning rate: actually we want the lr to be adjust to a figure that can make the function converge quickly and can also jump out of local minimals.
- Momemtm, 动量，滑动平均值

In [ ]:
label = None # the real class of the sample
predict = None # the answer of the classifer / module
# loss function
loss = tf.reduce_mean(tf.square(predict-label))
loss_cross = - tf.reduce_mean(label * tf.log(predict))
# optimizer
optim = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
# softmax
cross_with_softmax = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=predict, labels=tf.arg_max(label, 1))
c_w_s_mean = tf.reduce_mean(cross_with_softmax)
# the decay learning rate
global_step = tf.Variable(0, trainable=False)
learning_rate_decay = tf.train.exponential_decay(
    learning_rate=0.01, # base learning rate
    global_step=global_step, # count how many iterations of batch train is conducted, start from 0.
    decay_steps=5, # the learning rate decay would happen after how many times of batch training
    decay_rate=0.9, # the speed of decay 
    staircase=True
)
# Momemtm
ema = tf.train.ExponentialMovingAverage(
    decay=0.99,
    num_updates=global_step # the current iteration time
)
# apply method to all the changable parameters
ema_op = ema.apply(tf.trainable_variables())
with tf.control_dependencies(control_inputs=[train_Step, ema_op]):
    train_op = tf.no_op(name='train')
# check target parameter & run the ema_op
ema.average(var=para_name)
tf.Session.run(ema_op)

## Regularzation in tensorflow
- l1 norm: tf.contrib.layers.l1_regularizer()
- l2 nrom: tf.contrib.layers.l2_regularizer()
- collect all the losses to a certain list
- add the collected list data to a exist list

In [ ]:
tf.contrib.layers.l1_regularizer()
tf.contrib.layers.l2_regularizer()
# collect all the losses
tf.add_to_collection('loss_list', tf.contrib.layers.l1_regularizer(regularizer)(parameter))
loss_list = loss + tf.add_n(tf.get_collection('losses'))

## How to use a dataset: MNIST as an example
- this dataset is intergrated in tensorflow lib: train as training set and t10k as test set

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./dataset/Minst', one_hot=True) # use one-hot coding to encode the mnist dataset
mnist.train.num_examples # the size of the training set
mnist.validation.num_examples 
mnist.test.num_examples 
mnist.train.labels[0]
mnist.train.images[0]
batch_sample, batch_label = mnist.train.next_batch(BATCH_SIZE) # ectract a batch of data from mnist

## Some normal operations on data

In [ ]:
import os
tf.get_collection("collection_name") # get all the variables out as a list from a collection
tf.add_n([]) # add a list in correspond sptial position
tf.cast(sample, dtype) # transfer data type
tf.arg_max(sample, axis) # find the index of the max value at certain dimension
os.path.join("home", "name") # return home/name
moudle_saver = tf.train.Saver()
moudle_saver.save(tf.Session(), os.path.join("save_path", "moudle_name"), global_step=global_step) # save the current moudle to a target place
check_point = tf.train.get_checkpoint_state("path")
if check_point and check_point.model_checkpoint_path:
    moudle_saver.restore(tf.Session(), check_point.model_checkpoint_path) # restore the moudle back to a session
ema = tf.train.ExponentialMovingAverage(decay=0.9)
ema_restore = ema.variables_to_restore() # restore the parameters for exponetial moving average
ema_saver = tf.train.Saver(ema_restore)
# caculate the accuracy
correct_prediction = tf.equal(tf.argmax(label, 1), tf.argmax(predict, 1))
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## A simple training example
### Main functions
- forward(train_sample, regularizer): 前向传播，定义网络框架
- getWeight(shape, regularizer): 初始化权重，正则化权重
- getBias(shape): 根据每层中神经元个数初始化偏置
- backward(): 反向传播，在每个/批epoch中执行，通过loss和梯度优化参数
### Other key components
- regularization: 正则标准，是否使用
- learning rate: 是否decay
- optimizer: 配置优化器及其参数
- ema: 滑动平均，对参数优化时，考虑过去时间点上发生变化的梯度
- global_step: trainable=False，用于统计全局迭代次数情况
- with tf.Session() as sess: 进入训练循环，利用sess执行命令

In [ ]:
# import the packages and libs, then set tensorflow to be running in the eager mode
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt 
# tf.enable_eager_execution()

# define the hyperparameters and random generate the dataset
BATCH_SIZE = 30
SEED = 2 # fix the generate result (dataset)
EPOCH = 40000 # how many iterations we want

# prepare the dataset
rand = np.random.RandomState(SEED)
training_data = rand.randn(300, 2) # 300 sample matrix with 2 dimensions
training_label = [int((training_data_0 * training_data_0 + training_data_1 * training_data_1) < 2 for (training_data_0, training_data_1) in training_data)]
training_label_content = [['red' if label else 'blue'] for label in training_label]
# reshpe the training dataset
training_data = np.vstack(training_data).reshape(-1, 2)
training_label = np.vstack(training_label).reshape(-1, 1)
# use the training_label_content to plot the generated training samples
plt.scatter(training_data[:,0], training_data[:,1], c=np.squeeze(training_label_content))
plt.show()

# define the network
def getWeight(shape, regularizer):
    # randomly init the weight from a normal distribution
    weight = tf.Variable(tf.random_normal(shape), dtype=tf.float32)
    # define the regularization
    tf.add_to_collection('loss_list', tf.contrib.layers.l2_regularizer(regularizer)(weight))
    return weight

def getBias(shape):
    # init the bias for the network neruals
    bias = tf.Variable(tf.constant(0.01, shape=shape))
    return bias

# define the shape of the input variables
train_sample = tf.placeholder(tf.float32, shape=(None, 2)) # matrix of 2 dimensions with no number limitataion
train_label = tf.placeholder(tf.float32, shape=(None, 1))

# forward broadcasting through the first & second linear layer
weight_1 = getWeight([2, 11], 0.01)
bias_1 = getBias([11])
predict_1 = tf.nn.relu(tf.matmul(train_sample, weight_1) + bias_1)
weight_2 = getWeight([11, 1], 0.01)
bias_2 = getBias([1])
predict_2 = tf.matmul(predict_1, weight_2) + bias_2

# define loss function and our optimizer
loss_mse = tf.reduce_mean(tf.square(label - predict_2))
loss_list = loss_mse + tf.add_n(tf.get_collection('losslist'))
train_optim = tf.train.AdamOptimizer(0.0001).minimize(loss=loss_mse) # not using the l2 regularziation
train_optim_reg = tf.train.AdamOptimizer(0.0001).minimize(loss=loss_list)

# excute our iterations, this time we ignore the l2 regularization
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    for epoch in range(EPOCH):
        # every epoch we grab a batch of sample data and their labels, put into our optimizer
        start = (epoch + BATCH_SIZE) % 300
        end = start + BATCH_SIZE
        sess.run(train_optim, feed_dict={train_sample:training_data[start: end], train_label:training_label[start: end]})
        if epoch % 2000 == 0:
            loss_mse_v = sess.run(loss_mse, feed_dict={train_sample:training_data, train_label:training_label})
            print("At epoch: " + epoch + ", losss is: " + loss_mse_v)
    # 在(-3,3)区间步长0.01生成二位网格区间，并且拉直构成二列矩阵，本质为坐标点集合
    xx, yy = np.mgrid[-3:3:.01, -3:3:.01]
    grid = np.c_[xx.ravel(), yy.ravel()]
    # show the final moudle parameters
    predict = sess.run(train_label, feed_dict={train_sample:grid})
    predict = predict.reshape(xx.shape)
    print("Weight of 1 layer:" + sess.run(weight_1))
    print("Bias of 1 layer:" + sess.run(bias_1))
    print("Weight of 2 layer:" + sess.run(weight_2))
    print("Bias of 2 layer:" + sess.run(bias_2))

plt.scatter(training_data[:,0], training_data[:,1], c=np.squeeze(training_label_content))
plt.contour(xx, yy, probs, levels=[.5])
plt.show()

# excute another network, this time we use the l2 regularization
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    for epoch in range(EPOCH):
        # every epoch we grab a batch of sample data and their labels, put into our optimizer
        start = (epoch + BATCH_SIZE) % 300
        end = start + BATCH_SIZE
        sess.run(train_optim_reg, feed_dict={train_sample:training_data[start: end], train_label:training_label[start: end]})
        if epoch % 2000 == 0:
            loss_mse_v = sess.run(loss_mse, feed_dict={train_sample:training_data, train_label:training_label})
            print("At epoch: " + epoch + ", losss is: " + loss_mse_v)
    # 在(-3,3)区间步长0.01生成二位网格区间，并且拉直构成二列矩阵，本质为坐标点集合
    xx, yy = np.mgrid[-3:3:.01, -3:3:.01]
    grid = np.c_[xx.ravel(), yy.ravel()]
    # show the final moudle parameters
    predict = sess.run(train_label, feed_dict={train_sample:grid})
    predict = predict.reshape(xx.shape)
    print("Weight of 1 layer:" + sess.run(weight_1))
    print("Bias of 1 layer:" + sess.run(bias_1))
    print("Weight of 2 layer:" + sess.run(weight_2))
    print("Bias of 2 layer:" + sess.run(bias_2))

plt.scatter(training_data[:,0], training_data[:,1], c=np.squeeze(training_label_content))
plt.contour(xx, yy, probs, levels=[.5])
plt.show()